In [7]:
membrane_e = {'A':0.125718,'C':0.0256954,'D':0.00371686,'E':0.00394872,'F':0.0912202,
              'G':0.0720512,'H':0.00508179,'I':0.132166,'K':0.00204625,'L':0.189386,
              'M':0.0412358,'N':0.00779492,'P':0.0135996,'Q':0.00548422,'R':0.00202563,
              'S':0.0456952,'T':0.0513129,'V':0.146299,'W':0.0159998,'Y':0.0195234}

inner_e = { 'A':0.0607744,'C':0.00972669,'D':0.0479493,'E':0.0547882,'F':0.0266003,
            'G':0.0740532,'H':0.0283636,'I':0.0329976,'K':0.101684,'L':0.0572548,
            'M':0.0376071,'N':0.0474972,'P':0.0543291,'Q':0.0511293,'R':0.102204,
            'S':0.0812413,'T':0.0589999,'V':0.0350916,'W':0.0135542,'Y':0.0241536}
    
outer_e = {'A':0.10829,'C':0.0011907,'D':0.00428622,'E':0.00478901,'F':0.0955676,
           'G':0.0652582,'H':0.0140235,'I':0.0946458,'K':0.00598281,'L':0.165969,
           'M':0.038532,'N':0.0153548,'P':0.0290406,'Q':0.0142883,'R':0.00969443,
           'S':0.0463647,'T':0.039902,'V':0.0995781,'W':0.0649984,'Y':0.0822437}

begin = { 'A':0.0,'C':0.0,'D':0.0,'E':0.0,'F':0.0,
          'G':0.0,'H':0.0,'I':0.0,'K':0.0,'L':0.0,
          'M':0.0,'N':0.0,'P':0.0,'Q':0.0,'R':0.0,
          'S':0.0,'T':0.0,'V':0.0,'W':0.0,'Y':0.0}

emission = [begin,inner_e,membrane_e,outer_e,membrane_e]

b = 0.02
a = 1-b
transition = [[0,0.5,0,0.5,0],    # begin
             [0,a,b,0.0,0],       # inner
             [0,0,a,b,0.0],       # membrane - i->o
             [0,0,0,a,b],         # outer
             [0,b,0,0,a]          # membrane o->i
             ]



In [8]:
import numpy as np
def viterbi_decode(sequence,emission,transition):
    N = len(emission)
    L = len(sequence)
    V = np.zeros((N,L+1))       # An m*n matrix, initiated with 0's
    ptr = np.zeros((N,L+1))       # An m*n matrix, initiated with 0's
    V[0,0] = 1.0
    for i in range(1,L+1):
        for to_ in range(N):
            mv = .0
            for from_ in range(N):
                if mv < V[from_,i-1]*transition[from_][to_]:
                    mv = V[from_,i-1]*transition[from_][to_]
                    ptr[to_,i] = from_
            V[to_,i] = mv * emission[to_][sequence[i-1]]
    path_str = ""
    # Find maximal last step
    mv,p = 0.0,-1
    for to_ in range(N):
        if V[to_,L] > mv:
          mv, p = V[to_,L],int(to_) 
    for i in range(L,0,-1):
        path_str = str(int(p)) + path_str
        p = int(ptr[p,i])
    chunk = 80
    for ix in range(0,L+chunk-1,chunk):
        print(sequence[ix:ix+chunk])
        print(path_str[ix:ix+chunk])
        print()
    #print(ptr)
    #print(V)
            
            


In [9]:
viterbi_decode("FFFFFFFFFVVVVVVVVVVVVVVVVVVVVVVVVVVVVRRRRRRRRRRRRRRRRR",emission,transition)

FFFFFFFFFVVVVVVVVVVVVVVVVVVVVVVVVVVVVRRRRRRRRRRRRRRRRR
333333333444444444444444444444444444411111111111111111






In [11]:
viterbi_decode("MNGTEGPNFYVPFSNKTGVVRSPFEAPQYYLAEPWQFSMLAAYMFLLIVLGFPINFLTLYVTVQHKKLRTPLNYILLNLAVADLFMVFGGFTTTLYTSLHGYFVFGPTGCNLEGFFATLGGEIALWSLVVLAIERYVVVCKPMSNFRFGENHAIMGVAFTWVMALACAAPPLVGWSRYIPQGMQCSCGALYFTLKPEINNESFVIYMFVVHFSIPLIVIFFCYGQLVFTVKEAAAQQQESATTQKAEKEVTRMVIIMVIAFLICWLPYAGVAFYIFTHQGSDFGPIFMTIPAFFAKSSSVYNPVIYIMMNKQFRNCMLTTLCCGKNPLGDDEASTTVSKTETSQVAPA",emission,transition)

MNGTEGPNFYVPFSNKTGVVRSPFEAPQYYLAEPWQFSMLAAYMFLLIVLGFPINFLTLYVTVQHKKLRTPLNYILLNLA
00000000000000000000000000000000000000000000000000000000000000000000000000000000

VADLFMVFGGFTTTLYTSLHGYFVFGPTGCNLEGFFATLGGEIALWSLVVLAIERYVVVCKPMSNFRFGENHAIMGVAFT
00000000000000000000000000000000000000000000000000000000000000000000000000000000

WVMALACAAPPLVGWSRYIPQGMQCSCGALYFTLKPEINNESFVIYMFVVHFSIPLIVIFFCYGQLVFTVKEAAAQQQES
00000000000000000000000000000000000000000000000000000000000000000000000000000000

ATTQKAEKEVTRMVIIMVIAFLICWLPYAGVAFYIFTHQGSDFGPIFMTIPAFFAKSSSVYNPVIYIMMNKQFRNCMLTT
00000000000000000000000000000000000000000000000000000000000000000000000000000000

LCCGKNPLGDDEASTTVSKTETSQVAPA
000000000000000000000000000-1






In [12]:
import numpy as np
def log_viterbi_decode(sequence,emission,transition):
    N = len(emission)
    L = len(sequence)
    V = np.zeros((N,L+1))       # An m*n matrix, initiated with 0's
    ptr = np.zeros((N,L+1))       # An m*n matrix, initiated with 0's
    for s in range(1,N):
        V[s,0] = -np.inf
    for i in range(1,L+1):
        for to_ in range(N):
            mv = -np.inf
            for from_ in range(N):
                if mv < V[from_,i-1] + np.log(transition[from_][to_]):
                    mv = V[from_,i-1] + np.log(transition[from_][to_])
                    ptr[to_,i] = from_
            V[to_,i] = mv + np.log(emission[to_][sequence[i-1]])
    path_str = ""
    # Find maximal last step
    mv,p = -np.inf,-1
    for to_ in range(N):
        if V[to_,L] > mv:
          mv, p = V[to_,L],int(to_) 
    for i in range(L,0,-1):
        path_str = str(int(p)) + path_str
        p = int(ptr[p,i])
    chunk = 80
    for ix in range(0,L+chunk-1,chunk):
        print(sequence[ix:ix+chunk])
        print(path_str[ix:ix+chunk])
        print()


In [13]:
# >OPSD_SHEEP P02700 RHODOPSIN.
# MNGTEGPNFYVPFSNKTGVVRSPFEAPQYYLAEPWQFSMLAAYMFLLIVLGFPINFLTLY
# VTVQHKKLRTPLNYILLNLAVADLFMVFGGFTTTLYTSLHGYFVFGPTGCNLEGFFATLG
# GEIALWSLVVLAIERYVVVCKPMSNFRFGENHAIMGVAFTWVMALACAAPPLVGWSRYIP
# QGMQCSCGALYFTLKPEINNESFVIYMFVVHFSIPLIVIFFCYGQLVFTVKEAAAQQQES
# ATTQKAEKEVTRMVIIMVIAFLICWLPYAGVAFYIFTHQGSDFGPIFMTIPAFFAKSSSV
# YNPVIYIMMNKQFRNCMLTTLCCGKNPLGDDEASTTVSKTETSQVAPA

log_viterbi_decode("MNGTEGPNFYVPFSNKTGVVRSPFEAPQYYLAEPWQFSMLAAYMFLLIVLGFPINFLTLYVTVQHKKLRTPLNYILLNLAVADLFMVFGGFTTTLYTSLHGYFVFGPTGCNLEGFFATLGGEIALWSLVVLAIERYVVVCKPMSNFRFGENHAIMGVAFTWVMALACAAPPLVGWSRYIPQGMQCSCGALYFTLKPEINNESFVIYMFVVHFSIPLIVIFFCYGQLVFTVKEAAAQQQESATTQKAEKEVTRMVIIMVIAFLICWLPYAGVAFYIFTHQGSDFGPIFMTIPAFFAKSSSVYNPVIYIMMNKQFRNCMLTTLCCGKNPLGDDEASTTVSKTETSQVAPA",emission,transition)

MNGTEGPNFYVPFSNKTGVVRSPFEAPQYYLAEPWQFSMLAAYMFLLIVLGFPINFLTLYVTVQHKKLRTPLNYILLNLA
11111111111111111111111111111111111122222233333333333333333333333333333333333333

VADLFMVFGGFTTTLYTSLHGYFVFGPTGCNLEGFFATLGGEIALWSLVVLAIERYVVVCKPMSNFRFGENHAIMGVAFT
33333333333333333333333333344444444444444444444444444111111111111111111122222222

WVMALACAAPPLVGWSRYIPQGMQCSCGALYFTLKPEINNESFVIYMFVVHFSIPLIVIFFCYGQLVFTVKEAAAQQQES
22222222233333333333333344444444441111111122222222222222222222333444441111111111

ATTQKAEKEVTRMVIIMVIAFLICWLPYAGVAFYIFTHQGSDFGPIFMTIPAFFAKSSSVYNPVIYIMMNKQFRNCMLTT
11111111111122222222222233333333333333333333333333333333333333333344411111111111

LCCGKNPLGDDEASTTVSKTETSQVAPA
1111111111111111111111111111






//anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in log
  del sys.path[0]
//anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in log
  app.launch_new_instance()
